In [1]:
import pymysql as msql
import pandas as pd
import numpy as np

### pymysql.connect()方法参数
- host – Host where the database server is located
- user – Username to log in as
- password (passwd) – Password to use.
- database (db) – Database to use, None to not use a particular one.
- port – MySQL port to use, default is usually OK. (default: 3306)
- bind_address – When the client has multiple network interfaces, specify the interface from which to connect to the host. Argument can be a hostname or an IP address.
- charset – Charset you want to use.
- sql_mode – Default SQL_MODE to use.
- use_unicode – Whether or not to default to unicode strings. This option defaults to true for Py3k.
- cursorclass – Custom cursor class to use. default: list, or DictCusor
- connect_timeout – Timeout before throwing an exception when connecting. (default: 10, min: 1, max: 31536000)
- read_default_group – Group to read from in the configuration file.
- autocommit – Autocommit mode. None means use server default. (default: False)
- local_infile – Boolean to enable the use of LOAD DATA LOCAL command. (default: False)
- max_allowed_packet – Max size of packet sent to server in bytes. (default: 16MB) Only used to limit size of “LOAD LOCAL INFILE” data packet smaller than default (16KB).
- defer_connect – Don’t explicitly connect on contruction - wait for connect call. (default: False)

In [2]:
configs = {'host': 'localhost',
           'port': 3306,
           'user': 'root',
           'password': 'xiaoyu1986',
           'db': 'purchase',
           'use_unicode': True, 
           'charset': "utf8"}
          #'cursorclass': msql.cursors.DictCursor}  # cusorclass参数可不提供，此时返回列表查询结果 

In [3]:
conn = msql.connect(**configs)  # 等价于 msql.connect('localhost', 'root', 'xiaoyu1986', 'univeristy', 3306)

### connect对象的方法

- autocommit_mode = None.
    specified autocommit mode. None means use server default.

- begin().
    Begin transaction.

- close().
    Send the quit message and close the socket

- commit().
    Commit changes to stable storage

- cursor(cursor=None).
    Create a new cursor to execute queries with

- ping(reconnect=True).
    Check if the server is alive
    
- rollback().
    Roll back the current transaction

- select_db(db).
    Set current db

- show_warnings().
    SHOW WARNINGS

In [ ]:
conn.select_db('temple')  # 变更当前数据库

In [ ]:
conn.show_warnings()  # 提示信息

### cursor对象

- callproc(procname, args=()).
    - Execute stored procedure procname with args
    - procname – string, name of procedure to execute on server
    - args – Sequence of parameters to use with procedure
    - Returns the original args.

- close()
    - Closing a cursor just exhausts all remaining data.
 
- execute(query, args=None)
    - Execute a query
    - Parameters:	
        query (str) – Query to execute.
        args (tuple, list or dict) – parameters used with query. (optional)
    - Returns: Number of affected rows
    - Return type: int
    - If args is a list or tuple, %s can be used as a placeholder in the query. If args is a dict, %(name)s can be used as a placeholder in the query.
    
- executemany(query, args)
    - Run several data against one query
    - Parameters:	
        - query – query to execute on server
        - args – Sequence of sequences or mappings. It is used as parameter.
    - Returns:	
        - Number of rows affected, if any.   
    - This method improves performance on multiple-row INSERT and REPLACE. Otherwise it is equivalent to looping over args with execute().   
- fetchall()
    - Fetch all the rows
- fetchmany(size=None)
    - Fetch several rows
- fetchone()
    - Fetch the next row
- max_stmt_length = 1024000
    - Max statement size which executemany() generates.
    - Max size of allowed statement is max_allowed_packet - packet_header_size. Default value of max_allowed_packet is 1048576.

In [17]:
cursor = conn.cursor()  # 通过connect对象创建一个cursor对象

In [18]:
cursor.execute('select database()')

1

In [19]:
print(cursor.fetchall())

(('purchase',),)


### 创建表

- table_name: t1
- columns: id, name

In [8]:
cursor.execute('SET CHARACTER_SET_CLIENT=utf8')

0

In [ ]:
cursor.execute('DROP TABLE Product')

In [ ]:
conn.commit()

### 读取csv文件
（1）product表

In [ ]:
sql = """CREATE TABLE `Product` (`Product_ID` varchar(5) CHARACTER SET utf8, 
                                 `Product_Name` varchar(100) CHARACTER SET utf8,
                                 `Product_Code` varchar(10) CHARACTER SET utf8,
                                 `Product_Place` varchar(255) CHARACTER SET utf8,
                                 `Product_Date` Date,
                                 `Price` decimal(10, 2),
                                 `Unit` char(5) CHARACTER SET utf8,
                                 `Detail` varchar(50) CHARACTER SET utf8,
                                 `SubSort_ID` char(5) CHARACTER SET utf8,
                                 `Sort_ID` char(2) CHARACTER SET utf8                          
                                 ) DEFAULT CHARACTER SET=utf8"""
cursor.execute(sql)  # 创建Product表

In [ ]:
f = pd.read_csv('/Users/xiaoyu/Downloads/Product/Product.csv', thousands=',', decimal='.',
                dtype={'product_ID': np.str, 'Product_Place': np.str,
                       'Product_Code': np.str, 'SubSort_ID': np.str, 'Sort_ID': np.str})

In [ ]:
list(f.iloc[13].values)

In [ ]:
f.head()

In [ ]:
list(f.iloc[5515].values)

In [ ]:
isinstance('x', str)

In [ ]:
sql = """INSERT INTO `Product`(`Product_ID`, 
                               `Product_Name`, 
                               `Product_Code`,
                               `Product_Place`,
                               `Product_Date`,
                               `Price`,
                               `Unit`,
                               `Detail`,
                               `SubSort_ID`,
                               `Sort_ID`) 
         values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for i in range(5515,len(f)):
    print(i)
    data = f.iloc[i].values
    insert = []
    for j, x in enumerate(data): 
        if not isinstance(x, str):    
            if np.isnan(x):
                x = None
        else:  
            if j == 5:
                x = float(x)
            
        insert.append(x)
            
    cursor.execute(sql, insert)

In [ ]:
conn.commit()

In [ ]:
cursor.execute('select * from product where product_id < 50')

In [ ]:
for x in cursor:
    print(x)

####  (2) Member

In [10]:
sql_member = """CREATE TABLE `Member` (`User_name` varchar(50) CHARACTER SET utf8, 
                                 `User_password` varchar(100) CHARACTER SET utf8,
                                 `True_name` varchar(50) CHARACTER SET utf8,
                                 `Sex` enum('男', '女') CHARACTER SET utf8,
                                 `Phone` char(8) CHARACTER SET utf8,
                                 `Mobile` char(11) CHARACTER SET utf8,
                                 `Email` char(50) CHARACTER SET utf8,
                                 `Address` varchar(255) CHARACTER SET utf8,
                                 `Attribute` varchar(255) CHARACTER SET utf8                        
                                 ) DEFAULT CHARACTER SET=utf8"""

cursor.execute(sql_member)

0

In [16]:
f = pd.read_csv('/Users/xiaoyu/Downloads/sales system/Member.csv',
                dtype={'User_name': np.str, 'User_password': np.str,
                       'True_name': np.str, 'Sex': np.str, 'phone': np.str,
                       'Mobile': np.str, 'Email:': np.str, 'Address': np.str, 'Attribute': np.str})

In [17]:
f.head()

,User_name,User_password,True_name,Sex,phone,Mobile,Email,Address,Attribute
0,16号,admin123,周步新,女,69342295,13311777768,sffice@gmechina.com,新疆乌鲁木齐市团结路78号,NaN
1,Aya心冷,Cnhuker-Ker,张文倩,女,63127310,13061780039,xuyn@21stc.com.cn,湖北省咸宁市永安大道71号,NaN
2,BABY衣,Q23E1X,袁佳丽,女,88797580,13898281589,weilili@163.com,山东潍坊市奎文区胜利东街288号,NaN
3,Casper心冷,*123789,陈雯琼,女,69342295,13510643330,office@bypc.com.cn,青海省共和县恰卜恰镇绿洲北路33号,NaN
4,爱飞,hack521,郑珮琪,女,62014051,18146681065,NaN,湖北省武汉民院路5号,NaN


In [19]:
sql = """INSERT INTO `Member`(`User_name`, 
                               `User_password`, 
                               `True_name`,
                               `Sex`,
                               `Phone`,
                               `Mobile`,
                               `Email`,
                               `Address`,
                               `Attribute`) 
         values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for i in range(len(f)):
    print(i)
    data = f.iloc[i].values
    insert = []
    for j, x in enumerate(data): 
        if not isinstance(x, str):    
            if np.isnan(x):
                x = None
            
        insert.append(x)
            
    cursor.execute(sql, insert)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [20]:
conn.commit()

#### (3) Order

In [23]:
sql_order = """CREATE TABLE `Order` (`Order_ID` varchar(50) CHARACTER SET utf8, 
                                 `Product_ID` varchar(5) CHARACTER SET utf8,
                                 `Quantity` int(8),
                                 `User_name` varchar(50) CHARACTER SET utf8,
                                 `Order_date` Date,
                                 `Consignee` varchar(50) CHARACTER SET utf8,
                                 `Delivery_address` varchar(255) CHARACTER SET utf8,
                                 `Phone` char(8) CHARACTER SET utf8,
                                 `Mobile` char(11) CHARACTER SET utf8,
                                 `Email` char(50) CHARACTER SET utf8,
                                 `Remark` varchar(255) CHARACTER SET utf8
                                 ) DEFAULT CHARACTER SET=utf8"""

cursor.execute(sql_order)

0

In [28]:
f = pd.read_csv('/Users/xiaoyu/Downloads/Order.csv', encoding='gbk',
                dtype={'Order_ID': np.str, 'Product_ID': np.str, 'Quantity':np.str,'Phone': np.str, 'Mobile':np.str,
                       'Email':np.str, 'Remark':np.str})

In [29]:
f.head(5)

,Order_ID,Product_ID,Quantity,User_Name,Order_Date,Consignee,Delivery_Address,Phone,Mobile,Email,Remark
0,1,1009,2,爱飞的鱼,2017/1/10,欧丽莎,湖北武汉珞瑜路,65812255,18370993409,NaN,NaN
1,2,4370,120,流浪的鱼,2017/1/13,郭高强,山东省青岛市宁夏路308号,87741896,18938686935,gqcc@rubliq.bta.net.cn,NaN
2,3,4017,15,古月神仙,2017/1/29,刘远帆,吉林省延吉市公园路105号,62561896,13061706430,NaN,NaN
3,4,4514,50,变相怪杰,2017/2/1,何寅飞,云南省昆明市环城北路420号,63405533,15149176204,NaN,NaN
4,5,1015,1,白鲨,2017/2/10,沈颖,甘肃兰州市七里河区龚家坪东路1号,89382980,13916819491,yxq@ls-china.com.cn,NaN


In [30]:
sql = """INSERT INTO `Order`(`Order_ID`, `Product_ID`, `Quantity`, `User_name`, `Order_date`, `Consignee`,
                             `Delivery_address`, `Phone`, `Mobile`, `Email`, `Remark`) 
         values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for i in range(len(f)):
    print(i)
    data = f.iloc[i].values
    insert = []
    for j, x in enumerate(data): 
        if not isinstance(x, str):    
            if np.isnan(x):
                x = None
            
        insert.append(x)
            
    cursor.execute(sql, insert)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [31]:
conn.commit()

#### (4) Sort

In [25]:
sql_sort = """CREATE TABLE `Sort` (`Sort_ID` char(2) CHARACTER SET utf8, 
                                    `Sort_name` varchar(50) CHARACTER SET utf8
                                   ) DEFAULT CHARACTER SET=utf8"""
cursor.execute(sql_sort)

0

In [26]:
f = pd.read_csv('/Users/xiaoyu/Downloads/sales system/Sort.csv',
                dtype={'Sort_ID': np.str, 'Sort_Name': np.str})

In [27]:
sql = """INSERT INTO `Sort`(`Sort_ID`, `Sort_name`) 
         values (%s, %s)"""

for i in range(len(f)):
    print(i)
    data = f.iloc[i].values
    insert = []
    for j, x in enumerate(data): 
        if not isinstance(x, str):    
            if np.isnan(x):
                x = None
            
        insert.append(x)
            
    cursor.execute(sql, insert)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [28]:
conn.commit()

#### (5) SubSort

In [29]:
sql_subsort = """CREATE TABLE `SubSort` (`SubSort_ID` char(5) CHARACTER SET utf8,
                                    `SubSort_name` varchar(50) CHARACTER SET utf8,
                                    `Sort_ID`char(2) CHARACTER SET utf8 
                                   ) DEFAULT CHARACTER SET=utf8"""
cursor.execute(sql_subsort)

0

In [30]:
f = pd.read_csv('/Users/xiaoyu/Downloads/sales system/SubSort.csv',
                dtype={'SubSort_ID': np.str, 'SubSort_Name': np.str, 'Sort_ID': np.str})

In [31]:
sql = """INSERT INTO `SubSort`(`SubSort_ID`, `SubSort_name`, `Sort_ID`) 
         values (%s, %s, %s)"""

for i in range(len(f)):
    print(i)
    data = f.iloc[i].values
    insert = []
    for j, x in enumerate(data): 
        if not isinstance(x, str):    
            if np.isnan(x):
                x = None
            
        insert.append(x)
            
    cursor.execute(sql, insert)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [32]:
conn.commit()

### 修改表
- 增加两个属性 gender, depart_no

In [ ]:
cursor = conn.cursor(cursor=msql.cursors.DictCursor)

In [ ]:
cursor.execute("show columns from t1")
print [(x['Field'], x['Type'], x['Key']) for x in cursor.fetchall()]

In [ ]:
cursor.execute("ALTER TABLE `t1` ADD `gender` char(1), ADD `depart_no` char(5)")

### 插入单行.execute()

In [ ]:
sql = "INSERT INTO `t1`(`id`, `name`) values (%s, %s)"
cursor.execute(sql, (1, 'HH'))  # 参数化查询
cursor.execute(sql, ('2', 'LC'))  # 类型自动转换
cursor.execute(sql % (3, "'LL'"))  # 字符串拼接 sql%('1', 'HH'), 注意最后拼接成的字符串与mysql中的语法一致

In [ ]:
sql_1 = "INSERT INTO `t1`(`id`, `name`) values (:0, :1)"  # 参数化查询：占位符的另一种写法
cursor.execute(sql, (4, 'HX'))

In [ ]:
sql_2 = "INSERT INTO `t1`(`id`, `name`) values ({0}, {1})"  # 新型字符串格式化1
cursor.execute(sql_2.format(repr(5), repr('DC')))

In [ ]:
sql_3 = "INSERT INTO `t1`(`id`, `name`) values ({id}, {name})" # 新型字符串格式化2
cursor.execute(sql_3.format(id=repr(6), name=repr('LL')))

### 插入多行.executemany()

In [ ]:
ins_list = [(7, 'HX'), (8, 'GC')]
cursor.executemany(sql, ins_list)

In [ ]:
conn.commit()

### 查询（返回1行或多行结果）

In [ ]:
cursor = conn.cursor(cursor=msql.cursors.DictCursor)  # DictCursor返回的查询结果为字典

In [ ]:
cursor.execute("SELECT `id`, `name` FROM `t1`")
cursor.rowcount  # 返回execute()方法影响的行数

In [ ]:
r1 = cursor.fetchone()
print r1

- 移动游标cursor.scroll()
    - cursor.scroll(-1, mode='relative') # 相对当前位置移动
    - cursor.scroll(1, mode='absolute') # 相对绝对位置移动

In [ ]:
cursor.scroll(0, mode='absolute') # 相对绝对位置移动, 初始位置为0
for i, x in enumerate(cursor.fetchall()):
    print i, x

### 更新表

In [ ]:
sql = """UPDATE `t1`
         SET `gender`='1', `depart_no`='10001'
         WHERE `id`=1
      """

cursor.execute(sql)

In [ ]:
cursor.execute("SELECT `id`, `name`, `gender` FROM `t1` WHERE `id`=1")

In [ ]:
#cursor.scroll(-1, mode='relative')
cursor.fetchone()

- cursor.description获取查询结果字段信息

In [ ]:
cursor.description

### 删除表

In [ ]:
cursor.execute("DROP TABLE `t1`")

# 案例：构建university数据库中的表结构，并输入实例数据

In [ ]:
conn.select_db('progrm')

In [ ]:
def table_struc(sql, conn=conn):
    with conn.cursor() as cursor:
        cursor.execute(sql)
        conn.commit()

In [ ]:
def insert_data(sql, data, conn=conn):
    try:
        with conn.cursor() as cursor:
            cursor.executemany(sql, data)
            conn.commit()
    
    except Exception, e:
        print e
        conn.rollback()   

### 1. classroom表

In [ ]:
with conn.cursor() as cursor:
    sql = """create table `classroom`(
                           `building` varchar(15),
                           `room_number` varchar(7),
                           `capacity` decimal(4, 0),
                            primary key (`building`, `room_number`))
          """

    cursor.execute(sql)
    conn.commit()

In [ ]:
data = [('Packard', '101', '500'),
       ('Painter', '514', '10'),
       ('Taylor', '3128', '70'),
       ('Watson', '100', '30'),
       ('Watson', '120', '50')]

In [ ]:
try:
    with conn.cursor() as cursor:
        sql = "INSERT INTO `classroom` (`building`, `room_number`, `capacity`) VALUES (%s, %s, %s)"
        for x in data:  # cursor.executemany(sql, data)
            cursor.execute(sql, x)

        conn.commit()
        
except:
    conn.rollback()

### 2. department表

In [ ]:
sql = """create table `department`(
                                `dept_name` varchar(20) primary key,
                                `building` varchar(15),
                                `budget` decimal(12, 2));"""

table_struc(sql)

In [ ]:
data = [('Biology', 'Watson', '90000'), 
        ('Comp. Sci.', 'Taylor', '100000'),
        ('Elec. Eng.', 'Taylor', '85000'), 
        ('Finance', 'Painter', '120000'),
        ('History', 'Painter', '50000'), 
        ('Music', 'Packward', '80000'),
        ('Physics', 'Watson', '70000')]

sql = "INSERT INTO `department` (`dept_name`, `building`, `budget`) VALUES (%s, %s, %s)"

insert_data(sql, data)

In [ ]:
with conn.cursor() as cursor:
    cursor.execute("update department set dept_name = 'Comp. Sci.' where dept_name = 'Comp. Sci'")
    conn.commit()

In [ ]:
with conn.cursor() as cursor:
    sql = "SELECT `*` FROM `department` WHERE `building`=%s"
    cursor.execute(sql, 'Taylor')
    result = cursor.fetchone()
    print result

### 3. instructor表

In [ ]:
sql = """create table `instructor`(
                            `ID` varchar(5) primary key,
                            `name` varchar(20),
                            `dept_name` varchar(20),
                            `salary` decimal(8,2),
                             foreign key (dept_name) references department (dept_name));
      """

table_struc(sql)

In [ ]:
data = [('10101', 'Srinivasan', 'Comp. Sci.', '65000'),
       ('12121', 'Wu', 'Finance', '90000'),
       ('15151', 'Mozart', 'Music', '40000'),
       ('22222', 'Einstein', 'Physics', '95000'),
       ('32343', 'EI Said', 'History', '60000'),
       ('33456', 'Gold', 'Physics', '87000'),
       ('45565', 'Katz', 'Comp. Sci.', '75000'),
       ('58583', 'Califieri', 'History', '62000'),
       ('76766', 'Crick', 'Biology', '72000'),
       ('76543', 'Singh', 'Finance', '80000'),
       ('83821', 'Brandt', 'Comp. Sci.', '92000'),
       ('98345', 'Kim', 'Elec. Eng.', '80000')]

sql = "INSERT INTO `instructor` (`ID`, `name`, `dept_name`, `salary`) VALUES (%s, %s, %s, %s)"
insert_data(sql, data)

### 4. course表

In [ ]:
sql = """create table `course`(
                        `course_id` varchar(7) primary key,
                        `title` varchar(50),
                        `dept_name` varchar(20),
                        `credits` decimal(2,0),
                        foreign key (`dept_name`) references `department`(`dept_name`))"""

table_struc(sql)

In [ ]:
data = [('BIO-101', 'Intro. to Biology', 'Biology', '4'),
       ('BIO-301', 'Genetics', 'Biology', '4'),
       ('BIO-399', 'Computational Biology', 'Biology', '3'),
       ('CS-101', 'Intro. to Computer Science', 'Comp. Sci.', '4'),
       ('CS-190', 'Game Design', 'Comp. Sci.', '4'),
       ('CS-315', 'Robotics', 'Comp. Sci.', '3'),
       ('CS-319', 'Image Processing', 'Comp. Sci.', '3'),
       ('CS-347', 'Database System Concepts', 'Comp. Sci.', '3'),
       ('EE-181', 'Intro. to Digital Systems', 'Elec. Eng.', '3'),
       ('FIN-201', 'Investment Banking', 'Finance', '3'),
       ('HIS-351', 'World History', 'History', '3'),
       ('MU-199', 'Music Video Production', 'Music', '3'),
       ('PHY-101', 'Physical Principles', 'Physics', '4')]

sql = "INSERT INTO `course` (`course_id`, `title`, `dept_name`, `credits`) VALUES (%s, %s, %s, %s)"

insert_data(sql, data)

### 5. section表

In [ ]:
sql = """create table `section`(
                         `course_id` varchar(7),
                         `sec_id` varchar(8),
                         `semester` varchar(6),
                         `year` decimal(4,0),
                         `building` varchar(15),
                         `room_number` varchar(7),
                         `time_slot_id` varchar(4),
                         primary key (`course_id`, `sec_id`, `semester`, `year`),
                         foreign key (`course_id`) references `course`(`course_id`))
       """

table_struc(sql)

In [ ]:
data = [('BIO-101', '1', 'Summer', '2009', 'Painter', '514', 'B'),
       ('BIO-301', '1', 'Summer', '2010', 'Painter', '514', 'A'),
       ('CS-101', '1', 'Fall', '2009', 'Packard', '101', 'H'),
       ('CS-101', '1', 'Spring', '2010', 'Packard', '101', 'F'),
       ('CS-190', '1', 'Spring', '2009', 'Taylor', '3128', 'E'),
       ('CS-190', '2', 'Spring', '2009', 'Taylor', '3128', 'A'),
       ('CS-315', '1', 'Spring', '2010', 'Watson', '120', 'D'),
       ('CS-319', '1', 'Spring', '2010', 'Watson', '100', 'B'),
       ('CS-319', '2', 'Spring', '2010', 'Taylor', '3128', 'C'),
       ('CS-347', '1', 'Fall', '2009', 'Taylor', '3128', 'A'),
       ('EE-181', '1', 'Spring', '2009', 'Taylor', '3128', 'C'),
       ('FIN-201', '1', 'Spring', '2010', 'Packard', '101', 'B'),
       ('HIS-351', '1', 'Spring', '2010', 'Painter', '514', 'C'),
       ('MU-199', '1', 'Spring', '2010', 'Packard', '101', 'D'),
       ('PHY-101', '1', 'Fall', '2009', 'Watson', '100', 'A')
       ]

sql = """INSERT INTO `section` (`course_id`, `sec_id`, `semester`, `year`, `building`, `room_number`, `time_slot_id`) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s)"""

insert_data(sql, data)

### 6. teaches表

In [ ]:
sql = """create table `teaches`(
                         `ID` varchar(5),
                         `course_id` varchar(7),
                         `sec_id` varchar(8),
                         `semester` varchar(6),
                         `year` decimal(4,0),
                         primary key (`ID`, `course_id`, `sec_id`, `semester`, `year`),
                         foreign key (`ID`) references `instructor`(`ID`),
                         foreign key (`course_id`, `sec_id`, `semester`, `year`) references 
                                         `section`(`course_id`, `sec_id`, `semester`, `year`))
                         """

table_struc(sql)

In [ ]:
data = [('10101', 'CS-101', '1', 'Fall', '2009'),
        ('10101', 'CS-315', '1', 'Spring', '2010'),
        ('10101', 'CS-347', '1', 'Fall', '2009'),
        ('12121', 'FIN-201', '1', 'Spring', '2010'),
        ('15151', 'MU-199', '1', 'Spring', '2010'),
        ('22222', 'PHY-101', '1', 'Fall', '2009'),
        ('32343', 'HIS-351', '1', 'Spring', '2010'),
        ('45565', 'CS-101', '1', 'Spring', '2010'),
        ('45565', 'CS-319', '1', 'Spring', '2010'),
        ('76766', 'BIO-101', '1', 'Summer', '2009'),
        ('76766', 'BIO-301', '1', 'Summer', '2010'),
        ('83821', 'CS-190', '1', 'Spring', '2009'),
        ('83821', 'CS-190', '2', 'Spring', '2009'),
        ('83821', 'CS-319', '2', 'Spring', '2010'),
        ('98345', 'EE-181', '1', 'Spring', '2009')
       ]

sql = """INSERT INTO `teaches` (`ID`, `course_id`, `sec_id`, `semester`, `year`) 
                    VALUES (%s, %s, %s, %s, %s)"""

insert_data(sql, data)

### 7. prereq表

In [ ]:
sql = """create table `prereq`(
                        `course_id` varchar(7) primary key,
                        `prereq_id` varchar(7),
                        foreign key (`prereq_id`) references `course`(`course_id`))
          """

table_struc(sql)

In [ ]:
data = [('BIO-301', 'BIO-101'),
       ('BIO-399', 'BIO-101'),
       ('CS-190', 'CS-101'),
       ('CS-315', 'CS-101'),
       ('CS-319', 'CS-101'),
       ('CS-347', 'CS-101'),
       ('EE-181', 'PHY-101')]

sql = "INSERT INTO `prereq` (`course_id`, `prereq_id`) VALUES (%s, %s)"
insert_data(sql, data)

### 8. student表

In [ ]:
sql = """create table `student`(
                         `ID` varchar(5) primary key,
                         `name` varchar(20) not null,
                         `dept_name` varchar(20),
                         `tot_cred` decimal(3, 0) check (tot_cred >= 0),
                         foreign key (`dept_name`) references `department` (`dept_name`) on delete set null)
                         """
table_struc(sql)

In [ ]:
data = [('00128', 'Zhang', 'Comp. Sci.', '102'),
       ('12345', 'Shankar', 'Comp. Sci.', '32'),
       ('19991', 'Brandt', 'History', '80'),
       ('23121', 'Chavez', 'Finance', '110'),
       ('44553', 'Peltier', 'Physics', '56'),
       ('45678', 'Levy', 'Physics', '46'),
       ('54321', 'Williams', 'Comp. Sci.', '54'),
       ('55739', 'Sanchez', 'Music', '38'),
       ('70557', 'Snow', 'Physics', '0'),
       ('76543', 'Brown', 'Comp. Sci.', '58'),
       ('76653', 'Aoi', 'Elec. Eng.', '60'),
       ('98765', 'Bourikas', 'Elec. Eng.', '98'),
       ('98988', 'Tanaka', 'Biology', '120')]

sql = "INSERT INTO `student` (`ID`, `name`, `dept_name`, `tot_cred`) VALUES (%s, %s, %s, %s)"
insert_data(sql, data)

### 9. takes表

In [ ]:
sql = """create table `takes`(
                   `ID` varchar(5),
                   `course_id` varchar(7),
                   `sec_id` varchar(8),
                   `semester` varchar(6),
                   `year` decimal(4, 0),
                   `grade` varchar(2),
                   primary key (`ID`, `course_id`, `sec_id`, `semester`, `year`),
                   foreign key (`course_id`, `sec_id`, `semester`, `year`) references 
                                  `section`(`course_id`, `sec_id`, `semester`, `year`) on delete cascade,
                   foreign key (`ID`) references `student`(`ID`) on delete cascade)
      """

table_struc(sql)

In [ ]:
data = [('00128','CS-101','1','Fall','2009','A'),
        ('00128','CS-347','1','Fall','2009','A-'),
        ('12345','CS-101','1','Fall','2009','C'),
        ('12345','CS-190','2','Spring','2009','A'),
        ('12345','CS-315','1','Spring','2010','A'),
        ('12345','CS-347','1','Fall','2009','A'),
        ('19991','HIS-351','1','Spring','2010','B'),
        ('23121','FIN-201','1','Spring','2010','C+'),
        ('44553','PHY-101','1','Fall','2009','B-'),
        ('45678','CS-101','1','Fall','2009','F'),
        ('45678','CS-101','1','Spring','2010','B+'),
        ('45678','CS-319','1','Spring','2010','B'),
        ('54321','CS-101','1','Fall','2009','A-'),
        ('54321','CS-190','2','Spring','2009','B+'),
        ('55739','MU-199','1','Spring','2010','A-'),
        ('76543','CS-101','1','Fall','2009','A'),
        ('76543','CS-319','2','Spring','2010','A'),
        ('76653','EE-181','1','Spring','2009','C'),
        ('98765','CS-101','1','Fall','2009','C-'),
        ('98765','CS-315','1','Spring','2010','B'),
        ('98988','BIO-101','1','Summer','2009','A'),
        ('98988','BIO-301','1','Summer','2010',None)]

sql = "INSERT INTO `takes` (`ID`, `course_id`, `sec_id`, `semester`, `year`, `grade`) VALUES (%s, %s, %s, %s, %s, %s)"
insert_data(sql, data)

### 10. advisor表

In [ ]:
sql = """create table `advisor`(
                     `s_ID` varchar(5) primary key,
                     `i_ID` varchar(5),
                     foreign key (`i_ID`) references `instructor`(`ID`) on delete set null,
                     foreign key (`s_ID`) references `student`(`ID`) on delete cascade)
      """

table_struc(sql)

In [ ]:
data = [('00128','45565'),
       ('12345','10101'),
       ('23121','76543'),
       ('44553','22222'),
       ('45678','22222'),
       ('76543','45565'),
       ('76653','98345'),
       ('98765','98345'),
       ('98988','76766')]

sql = "INSERT INTO `advisor` (`s_ID`, `i_ID`) VALUES (%s, %s)"
insert_data(sql, data)

### timeslot表

In [ ]:
sql = """create table `timeslot`(
                      `time_slot_id` varchar(4),
                      `day` varchar(4) check (day in ('M', 'T', 'W', 'R', 'F', 'S', 'U')),
                      `start_time` time,
                      `end_time` time,
                      primary key (`time_slot_id`, `day`, `start_time`))
      """

table_struc(sql)

In [ ]:
data = [('A','M','8:00','8:50'),
       ('A','W','8:00','8:50'),
       ('A','F','8:00','8:50'),
       ('B','M','9:00','9:50'),
       ('B','W','9:00','9:50'),
       ('B','F','9:00','9:50'),
       ('C','M','11:00','11:50'),
       ('C','W','11:00','11:50'),
       ('C','F','11:00','11:50'),
       ('D','M','13:00','13:50'),
       ('D','W','13:00','13:50'),
       ('D','F','13:00','13:50'),
       ('E','T','10:30','11:45'),
       ('E','R','10:30','11:45'),
       ('F','T','14:30','15:45'),
       ('F','R','14:30','15:45'),
       ('G','M','16:00','16:50'),
       ('G','W','16:00','16:50'),
       ('G','F','16:00','16:50'),
       ('H','W','10:00','12:30')]

sql = "INSERT INTO `timeslot` (`time_slot_id`, `day`, `start_time`, `end_time`) VALUES (%s, %s, %s, %s)"
insert_data(sql, data)

In [ ]:
with conn.cursor() as cursor:
    cursor.call_proc('')
    cursor.execute('call ..')
    
